# 数据分析及实践 实验二
#### 崔士强 PB22151743
下面每一个代码框对应一项任务，共5个.

In [6]:
import urllib
import re
import json

url = "https://dblp.dagstuhl.de/db/conf/kdd/kdd2023.html"
response = urllib.request.urlopen(url)

# 解码，获得字符串
html = response.read().decode("utf-8")

# 将字符串写入文件
with open("page.txt", "w") as file:
    file.write(html)

In [7]:
with open("page.txt", "r") as file:
    lines = file.readlines()
    
# 找到网站页面内容对应的行
for line in lines:
    if("Applied Data Track Full Papers" in line):
        i = lines.index(line)

# 标题格式
title_pattern = '<h2 id=.{1,100}>.{1,100}</h2></header>'

# 查找所有标题所在的语句
titleList = re.findall(title_pattern, lines[i])

# 逐个提取标题
for title in titleList:
    temp = re.sub('<h2 id=.+\">', '', title)
    print(re.sub('</h2></header>', '', temp))

Research Track Full Papers
Applied Data Track Full Papers
Hands On Tutorials
Lecture Style Tutorials
Workshop Summaries


In [8]:
strList = lines[i].split("<header class=\"h2\">")

track1dict = {}
track2dict = {}
trackDictList = [track1dict, track2dict]

track1dict["track"] = "Research Track Full Papers"
track1dict["papers"] = []
track2dict["track"] = "Applied Data Track Full Papers"
track2dict["papers"] = []

for track_no in range(1,3):
    # 以论文为单位初步分割
    temp_list = strList[track_no].split("<meta property=\"genre\" content=\"computer science\">")
    for str in temp_list:
        paper_dict = {}
        try:
            # 查找所有作者
            author_pattern = 'itemprop=\"name\" title=\"[a-zA-Z -]+\"'
            authorList = re.findall(author_pattern, str)
            paper_dict["author"] = [author.split('\"')[-2] for author in authorList]
            
            # 查找标题
            paper_dict["title"] = str.split("<span class=\"title\" itemprop=\"name\">")[1].split("</span>")[0]
            
            # 查找收录起始页和终止页
            paper_dict["startPage"] = int(str.split("<span itemprop=\"pagination\">")[1].split("-")[0])
            paper_dict["endPage"] = int(str.split("<span itemprop=\"pagination\">")[1].split("-")[1].split("</span>")[0])
            
            # 存进对应track的papers列表
            if(track_no == 1):
                track1dict["papers"].append(paper_dict)
            else:
                track2dict["papers"].append(paper_dict)
        except:
            continue
   
print(len(track1dict["papers"]))
print(len(track2dict["papers"]))

313
183


In [9]:
with open('kdd23.json', 'w', encoding='utf-8') as json_file:
    json.dump(trackDictList, json_file, indent=2)

In [10]:
def getPapers(str):
    html = urllib.request.urlopen(str).read().decode("utf-8")
    paper_dict_list = []
    content = html.split("class=\"hide-head h2\">")[2]
    str_list = content.split("<meta property=\"genre\" content=\"computer science\">")
    for str in str_list:
        try:
            paper_dict = {}
            authors = re.findall("<span itemprop=\"name\" title=\"[^\"]+\">[^<]+|<span class=\"this-person\" itemprop=\"name\">[^<]+", str)
            author_list = []
            for author in authors:
                author_list.append(author.split(">")[-1])
            paper_dict["authors"] = author_list
            
            title = re.search("<span class=\"title\" itemprop=\"name\">[^<]+", str).group().split(">")[-1]
            paper_dict["title"] = title
            
            info_name = re.search("<span itemprop=\"isPartOf\" itemscope itemtype=\"[^\"]+\"><span itemprop=\"name\">[^<]+", str).group().split(">")[-1]
            try:
                info_volume = re.search("<span itemprop=\"volumeNumber\">[^<]+", str).group().split(">")[-1]
                info_name = info_name + " " + info_volume
            except:
                pass
            try:
                info_issue = re.search("<span itemprop=\"issueNumber\">[^<]+", str).group().split(">")[-1]
                info_name = info_name + "(" + info_issue + ")"
            except:
                pass
            try:
                info_pagination = re.search("<span itemprop=\"pagination\">[^<]+", str).group().split(">")[-1]
                info_name = info_name + ": " + info_pagination
            except:
                pass
            paper_dict["publishInfo"] = info_name
                
            datePublished = re.search("<span itemprop=\"datePublished\">[0-9]{4}", str).group().split(">")[-1]
            paper_dict["year"] = datePublished
            
            paper_dict_list.append(paper_dict)
        except:
            continue
    return paper_dict_list

researcherDictList = []
researcher_pattern = '<span itemprop=\"author\" itemscope itemtype=\"http://schema.org/Person\"><a href=\"[^\"]+\" itemprop=\"url\"><span itemprop=\"name\" title=\"[^\"]+\">[^<]+</span></a><img src=\"[^\"]+\" style=\"padding-left:0.25em;\" alt=\"\" title=\"[^\"]{19}'
for track_no in range(1,3):
    # 截取前10篇论文
    temp_list = strList[track_no].split("<meta property=\"genre\" content=\"computer science\">")[0:10]
    for str in temp_list:
        try:
            # 找到每个包含研究者信息的字符串
            researcherList = re.findall(researcher_pattern, str)
            
            for res_str in researcherList:
                researcher_dict = {}
                name = re.search('itemprop=\"name\" title=\"[^\"]+\">[^<]+', res_str).group().split(">")[-1]
                
                # 查重
                hasDuplicate = False
                for dic in researcherDictList:
                    if(dic["researcher"] == name):
                        hasDuplicate = True
                        break
                if(hasDuplicate):
                    continue
                
                researcher_dict["researcher"] = name
                
                researcher_dict["orcID"] = re.search('[0-9-X]{19}', res_str).group()
                
                researcher_html = re.search('https://[^"]+\.html', res_str).group()
                
                researcher_dict["papers"] = getPapers(researcher_html)
                
                researcherDictList.append(researcher_dict)
        except:
            continue

with open('researchers.json', 'w', encoding='utf-8') as json_file:
    json.dump(researcherDictList, json_file, indent=2)

Florian Adriaens
Honglian Wang
Aristides Gionis
Rishi Advani
Paolo Papotti
Abolfazl Asudeh
Amine Allouah
Christian Kroer
Xuan Zhang
Vashist Avadhanula
Nona Bohanon
Anil Dania
Caner Gocmen
Sergey Pupyrev
Parikshit Shah
Nicol&#225;s Stier Moses
Ken Rodr&#237;guez Taarup
Mario Almagro
Emilio J. Almaz&#225;n
Diego Ortego
David Jim&#233;nez
Amel Awadelkarim
Arjun Seshadri
Itai Ashlagi
Irene Lo
Johan Ugander
Jiaxin Bai
Chen Luo
Zheng Li
Qingyu Yin
Bing Yin
Yangqiu Song
Ergute Bao
Dawei Gao
Xiaokui Xiao
Yaliang Li
Anna Beer
Andrew Draganov
Ellen Hohma
Philipp Jahn
Christian M. M. Frey
Ira Assent
Siddharth Bhatia
Mohit Wadhwa
Kenji Kawaguchi
Neil Shah
Philip S. Yu
Bryan Hooi
Adam Breuer
Nazanin Khosravani Tehrani
Michael Tingley
Bradford Cottel
Abhinav Anand
Surender Kumar
Nandeesh Kumar
Samir Shah
Wenxuan Ao
Guozhen Zhang
Yong Li
Depeng Jin
Yash Kumar Atri
Vikram Goyal
Tanmoy Chakraborty
Vasilisa Bashlovkina
Riley Matthews
Zhaobin Kuang
Simon Baumgartner
Michael Bendersky
Xi Jiang
Van Tran
Ar